# SUP ML 3 - PREDICT

# Librerias

In [38]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns',None)

import warnings
warnings.filterwarnings("ignore")

# Carga modelo

* Cargar el modelo
* Obtener la lista de model features

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
# Carga del modelo:
best_model = pd.read_pickle('/content/drive/MyDrive/NDS_2023/EJERCICIO-ML-Sup/model/classifier.pkl')

In [41]:
features_model =  list(best_model.feature_names_in_)
print('Estas son las variables que tenemos que conseguir en el preprocessing para que el modelo funcione:\n', features_model)

Estas son las variables que tenemos que conseguir en el preprocessing para que el modelo funcione:
 ['rev', 'mou', 'totmrc', 'da', 'ovrmou', 'ovrrev', 'vceovr', 'datovr', 'roam', 'change_mou', 'change_rev', 'drop_vce', 'drop_dat', 'blck_vce', 'blck_dat', 'unan_vce', 'unan_dat', 'plcd_vce', 'plcd_dat', 'recv_vce', 'recv_sms', 'comp_vce', 'comp_dat', 'custcare', 'ccrndmou', 'cc_mou', 'inonemin', 'threeway', 'mou_cvce', 'mou_cdat', 'mou_rvce', 'owylis_vce', 'mouowylisv', 'iwylis_vce', 'mouiwylisv', 'peak_vce', 'peak_dat', 'mou_peav', 'mou_pead', 'opk_vce', 'opk_dat', 'mou_opkv', 'mou_opkd', 'drop_blk', 'attempt', 'complete', 'callfwdv', 'callwait', 'months', 'uniqsubs', 'actvsubs', 'asl_flag', 'totcalls', 'totmou', 'totrev', 'adjrev', 'adjmou', 'adjqty', 'avgrev', 'avgmou', 'avgqty', 'avg3mou', 'avg3qty', 'avg3rev', 'avg6mou', 'avg6qty', 'avg6rev', 'refurb_new', 'hnd_price', 'phones', 'models', 'truck', 'rv', 'ownrent', 'lor', 'dwlltype', 'adults', 'income', 'numbcars', 'dwllsize', 'forgn

# Carga PREDICT dataset

In [42]:
# Carga de los datos del PREDICT:
df = pd.read_csv('/content/drive/MyDrive/NDS_2023/EJERCICIO-ML-Sup/data/telecom_churn_PREDICT.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 99 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rev               9777 non-null   float64
 1   mou               9777 non-null   float64
 2   totmrc            9777 non-null   float64
 3   da                9777 non-null   float64
 4   ovrmou            9777 non-null   float64
 5   ovrrev            9777 non-null   float64
 6   vceovr            9777 non-null   float64
 7   datovr            9777 non-null   float64
 8   roam              9777 non-null   float64
 9   change_mou        9704 non-null   float64
 10  change_rev        9704 non-null   float64
 11  drop_vce          10000 non-null  float64
 12  drop_dat          10000 non-null  float64
 13  blck_vce          10000 non-null  float64
 14  blck_dat          10000 non-null  float64
 15  unan_vce          10000 non-null  float64
 16  unan_dat          10000 non-null  float64

In [43]:
df.shape

(10000, 99)

# ML Preprocessing

Se deben realizar las mismas transformaciones que se aplicaron en el preprocessing del train:
  * Eliminar mismas variables
  * Imputar mismos valores a nulos
  * Aplicar mismo encoding de categoricos
  * ...todo lo necesario para replicar las variables con las que el modelo se entrenó

NO SE DEBEN repetir:
  * Analisis visual de los datos
  * Analisis de correlaciones
  * Analisis de varianza
  * Calculo de metricas que varien con la distribución

In [44]:
target = 'churn'

In [45]:
df = df.set_index('Customer_ID') # Definimos el 'Customer ID' como el indice de nuestro dataset

In [46]:
encoder_map = { # Vamos a encodear el resto de variables booleanas a través de .map()
    'asl_flag': {'Y': 1, 'N': 0},
    'refurb_new': {'R': 1, 'N': 0},
    'ownrent': {'O': 1, 'R': 0},
    'dwlltype': {'M': 1, 'S': 0},
    'kid0_2': {'Y': 1, 'U': 0},
    'kid3_5': {'Y': 1, 'U': 0},
    'kid6_10': {'Y': 1, 'U': 0},
    'kid11_15': {'Y': 1, 'U': 0},
    'kid16_17': {'Y': 1, 'U': 0},
    'creditcd': {'Y': 1, 'N': 0}}
for col in encoder_map.keys():
  df[col] = df[col].map(encoder_map[col])

area_mapping = {
    'NEW YORK CITY AREA': 'Northeast',
    'DC/MARYLAND/VIRGINIA AREA': 'Mid-Atlantic',
    'MIDWEST AREA': 'Midwest',
    'ATLANTIC SOUTH AREA': 'South',
    'CALIFORNIA NORTH AREA': 'West',
    'DALLAS AREA': 'South',
    'NEW ENGLAND AREA': 'Northeast',
    'SOUTHWEST AREA': 'South',
    'CHICAGO AREA': 'Midwest',
    'LOS ANGELES AREA': 'West',
    'GREAT LAKES AREA': 'Midwest',
    'OHIO AREA': 'Midwest',
    'NORTHWEST/ROCKY MOUNTAIN AREA': 'West',
    'NORTH FLORIDA AREA': 'South',
    'CENTRAL/SOUTH TEXAS AREA': 'South',
    'HOUSTON AREA': 'South',
    'SOUTH FLORIDA AREA': 'South',
    'TENNESSEE AREA': 'South',
    'PHILADELPHIA AREA': 'Mid-Atlantic',
    'Unknown': 'Unknown'
    }
df['area_group'] = df['area'].map(area_mapping)

In [47]:
df.drop(['area'], axis=1, inplace=True)

In [48]:
def obtener_lista_variables(dataset):
    lista_numericas = []
    lista_boolean = []
    lista_categoricas = []

    for i in dataset:
        if (dataset[i].dtype.kind == "f" or dataset[i].dtype.kind == "i") and len(dataset[i].unique()) != 2 and i != target:
            lista_numericas.append(i)
        elif (dataset[i].dtype.kind == "f" or dataset[i].dtype.kind == "i" or dataset[i].dtype.kind == "b") and len(dataset[i].unique()) == 2 and i != target:
            lista_boolean.append(i)
        elif (dataset[i].dtype.kind == "O") and i != target:
            lista_categoricas.append(i)

    return lista_numericas, lista_boolean, lista_categoricas

In [49]:
lista_numericas, lista_boolean, lista_categoricas = obtener_lista_variables(df)
print(lista_numericas)
print(lista_boolean)
print(lista_categoricas)

['rev', 'mou', 'totmrc', 'da', 'ovrmou', 'ovrrev', 'vceovr', 'datovr', 'roam', 'change_mou', 'change_rev', 'drop_vce', 'drop_dat', 'blck_vce', 'blck_dat', 'unan_vce', 'unan_dat', 'plcd_vce', 'plcd_dat', 'recv_vce', 'recv_sms', 'comp_vce', 'comp_dat', 'custcare', 'ccrndmou', 'cc_mou', 'inonemin', 'threeway', 'mou_cvce', 'mou_cdat', 'mou_rvce', 'owylis_vce', 'mouowylisv', 'iwylis_vce', 'mouiwylisv', 'peak_vce', 'peak_dat', 'mou_peav', 'mou_pead', 'opk_vce', 'opk_dat', 'mou_opkv', 'mou_opkd', 'drop_blk', 'attempt', 'complete', 'callfwdv', 'callwait', 'months', 'uniqsubs', 'actvsubs', 'totcalls', 'totmou', 'totrev', 'adjrev', 'adjmou', 'adjqty', 'avgrev', 'avgmou', 'avgqty', 'avg3mou', 'avg3qty', 'avg3rev', 'avg6mou', 'avg6qty', 'avg6rev', 'hnd_price', 'phones', 'models', 'truck', 'rv', 'ownrent', 'lor', 'dwlltype', 'adults', 'income', 'numbcars', 'forgntvl', 'kid0_2', 'kid3_5', 'kid6_10', 'kid11_15', 'kid16_17', 'creditcd', 'eqpdays']
['asl_flag', 'refurb_new']
['new_cell', 'crclscod', 'p

In [50]:
imputar_nulos = {
    **{col: -999999999 for col in lista_numericas},
    **{col: -999999999 for col in lista_boolean},
    **{col: 'Unknown' for col in lista_categoricas}
}
df = df.fillna(imputar_nulos)

In [51]:
dwelling_size_map = {
    'A': 1,
    'B': 2,
    'C': 3,
    'D': 4,
    'E': 5,
    'F': 6,
    'G': 7,
    'H': 8,
    'I': 9,
    'J': 10,
    'K': 11,
    'L': 12,
    'M': 13,
    'N': 14,
    'O': 15,
    'P': 16,
    'Unknown': -999999999
}
df['dwllsize'] = df['dwllsize'].map(dwelling_size_map)

In [52]:
# Carga del ohe:
ohe_loaded = pd.read_pickle('/content/drive/MyDrive/NDS_2023/EJERCICIO-ML-Sup/model/ohe_transformer.pkl')

In [53]:
ohe_cols = list(df.select_dtypes(include=['object','category']).columns)
df_ohe = ohe_loaded.transform(df[ohe_cols]).toarray()
df_ohe = pd.DataFrame(df_ohe, columns=ohe_loaded.get_feature_names_out())

In [54]:
df_ohe.index = df.index
print(df_ohe.columns)

Index(['new_cell_N', 'new_cell_U', 'new_cell_Y', 'crclscod_A', 'crclscod_AA',
       'crclscod_BA', 'crclscod_CA', 'crclscod_EA',
       'crclscod_infrequent_sklearn', 'prizm_social_one_C',
       'prizm_social_one_S', 'prizm_social_one_T', 'prizm_social_one_U',
       'prizm_social_one_Unknown', 'prizm_social_one_infrequent_sklearn',
       'dualband_N', 'dualband_T', 'dualband_Y', 'dualband_infrequent_sklearn',
       'hnd_webcap_Unknown', 'hnd_webcap_WC', 'hnd_webcap_WCMB',
       'hnd_webcap_infrequent_sklearn', 'marital_A', 'marital_B', 'marital_M',
       'marital_S', 'marital_U', 'marital_infrequent_sklearn',
       'infobase_Unknown', 'HHstatin_A', 'HHstatin_B', 'HHstatin_C',
       'HHstatin_I', 'HHstatin_Unknown', 'HHstatin_infrequent_sklearn',
       'ethnic_G', 'ethnic_H', 'ethnic_N', 'ethnic_S', 'ethnic_U',
       'ethnic_infrequent_sklearn', 'area_group_Mid-Atlantic',
       'area_group_Midwest', 'area_group_Northeast', 'area_group_South',
       'area_group_West', 'area_

In [55]:
df.drop(columns=ohe_cols, inplace=True)

In [56]:
df = pd.concat([df, df_ohe], axis=1)
print(df.columns)

Index(['rev', 'mou', 'totmrc', 'da', 'ovrmou', 'ovrrev', 'vceovr', 'datovr',
       'roam', 'change_mou',
       ...
       'ethnic_N', 'ethnic_S', 'ethnic_U', 'ethnic_infrequent_sklearn',
       'area_group_Mid-Atlantic', 'area_group_Midwest', 'area_group_Northeast',
       'area_group_South', 'area_group_West', 'area_group_infrequent_sklearn'],
      dtype='object', length=136)


In [57]:
features = [i for i in df.columns if i != target]
X = df[features]

In [58]:
# Eliminamos las features altamente correlacionadas
drop_cols = ['rev', 'rev', 'totmrc', 'da', 'da', 'da', 'ovrmou', 'ovrmou', 'ovrmou', 'ovrmou', 'ovrrev', 'ovrrev', 'ovrrev', 'ovrrev', 'ovrmou', 'vceovr', 'vceovr', 'vceovr', 'vceovr', 'ovrmou', 'ovrrev', 'datovr', 'datovr',
             'datovr', 'datovr', 'ovrmou', 'ovrrev', 'vceovr', 'roam', 'roam', 'roam', 'roam', 'ovrmou', 'ovrrev', 'vceovr', 'roam', 'change_rev', 'plcd_vce', 'comp_dat', 'cc_mou', 'inonemin', 'mou_opkd', 'plcd_vce', 'attempt',
             'plcd_vce', 'comp_vce', 'attempt', 'totrev', 'totmou', 'totcalls', 'avg6mou', 'avg6mou', 'avg6qty', 'phones', 'models', 'phones', 'truck', 'truck', 'forgntvl', 'truck', 'rv', 'forgntvl', 'truck', 'rv', 'forgntvl',
             'kid3_5', 'truck', 'rv', 'forgntvl', 'kid6_10', 'kid6_10', 'truck', 'kid11_15', 'forgntvl', 'kid11_15', 'kid11_15', 'kid11_15', 'truck', 'rv', 'forgntvl', 'kid16_17', 'kid3_5', 'kid6_10', 'kid11_15', 'creditcd',
             'creditcd', 'creditcd', 'creditcd', 'creditcd', 'creditcd', 'creditcd', 'creditcd', 'refurb_new', 'phones', 'models', 'hnd_webcap_infrequent_sklearn', 'truck', 'rv', 'forgntvl', 'marital_infrequent_sklearn',
             'marital_infrequent_sklearn', 'kid6_10', 'kid11_15', 'marital_infrequent_sklearn', 'creditcd', 'adults']
X.drop(columns = drop_cols, inplace=True)
print('Eliminadas columnas altamente correlacionadas:', drop_cols)

Eliminadas columnas altamente correlacionadas: ['rev', 'rev', 'totmrc', 'da', 'da', 'da', 'ovrmou', 'ovrmou', 'ovrmou', 'ovrmou', 'ovrrev', 'ovrrev', 'ovrrev', 'ovrrev', 'ovrmou', 'vceovr', 'vceovr', 'vceovr', 'vceovr', 'ovrmou', 'ovrrev', 'datovr', 'datovr', 'datovr', 'datovr', 'ovrmou', 'ovrrev', 'vceovr', 'roam', 'roam', 'roam', 'roam', 'ovrmou', 'ovrrev', 'vceovr', 'roam', 'change_rev', 'plcd_vce', 'comp_dat', 'cc_mou', 'inonemin', 'mou_opkd', 'plcd_vce', 'attempt', 'plcd_vce', 'comp_vce', 'attempt', 'totrev', 'totmou', 'totcalls', 'avg6mou', 'avg6mou', 'avg6qty', 'phones', 'models', 'phones', 'truck', 'truck', 'forgntvl', 'truck', 'rv', 'forgntvl', 'truck', 'rv', 'forgntvl', 'kid3_5', 'truck', 'rv', 'forgntvl', 'kid6_10', 'kid6_10', 'truck', 'kid11_15', 'forgntvl', 'kid11_15', 'kid11_15', 'kid11_15', 'truck', 'rv', 'forgntvl', 'kid16_17', 'kid3_5', 'kid6_10', 'kid11_15', 'creditcd', 'creditcd', 'creditcd', 'creditcd', 'creditcd', 'creditcd', 'creditcd', 'creditcd', 'refurb_new', '

In [59]:
# Eliminamos columnas practicamente constantes (nula varianza)
cols_lowvar = ['dualband_infrequent_sklearn', 'area_group_infrequent_sklearn']
X.drop(columns=cols_lowvar,inplace=True)
print(len(cols_lowvar), ' low variance features were removed:\n', cols_lowvar)

2  low variance features were removed:
 ['dualband_infrequent_sklearn', 'area_group_infrequent_sklearn']


# Check model features

* Comprobar que tenemos en el dataset preprocesado todas las model features, de lo contrario no podremos hacer predict.
* Ordenar las variables en mismo orden que las model features

In [60]:
features_test = list(df.columns)

print('Columnas en dataset:',len(features_test))
print('Variables en modelos:',len(features_model))
print('¿Match?:', features_model == features_test)

Columnas en dataset: 136
Variables en modelos: 136
¿Match?: True


In [61]:
missing_features = [i for i in features_model if i not in features_test]
print('Variables que faltan en el dataset:\n', missing_features)

Variables que faltan en el dataset:
 []


In [62]:
for col in missing_features:
  df[col]= 0

In [63]:
drop_features = [i for i in features_test if i not in features_model]
print('Variables que debes eliminar de tu dataset:\n', drop_features)

Variables que debes eliminar de tu dataset:
 []


In [64]:
df.drop(columns = drop_features, inplace=True)

In [65]:
# Reordena variables
df = df[features_model]

In [66]:
features_test = list(df.columns)

print('Columnas en dataset:',len(features_test))
print('Variables en modelos:',len(features_model))
print('¿Match?:', features_model == features_test)

Columnas en dataset: 136
Variables en modelos: 136
¿Match?: True


# Rescaling

* Si se entrenó el modelo con un dataset estandarizado, estandarizar con mismo scaler.

In [67]:
# En este caso no es necesario rescalar porque no entrenamos con dataset reescalado

# PREDICT

* predict() y predict_proba()

In [68]:
predict = best_model.predict(df).astype(int)
predict

array([0, 0, 0, ..., 1, 1, 1])

In [69]:
predict_proba = best_model.predict_proba(df)[:,1]
predict_proba

array([0.39210647, 0.3777909 , 0.2190626 , ..., 0.94455029, 0.77411164,
       0.81506547])

# Guarda predicciones

* Guardar las predicciones en data path. Cada fila debe estar etiquetada con el ID.

In [70]:
predictions = pd.DataFrame(predict, columns=['churn'], index = df.index)

In [71]:
print(predictions)

             churn
Customer_ID       
1090001          0
1090002          0
1090003          0
1090004          1
1090005          0
...            ...
1099996          1
1099997          1
1099998          1
1099999          1
1100000          1

[10000 rows x 1 columns]


In [72]:
predictions.value_counts()

churn
0        7454
1        2546
dtype: int64

In [35]:
predictions.to_csv('/content/drive/MyDrive/NDS_2023/EJERCICIO-ML-Sup/model/predictions.csv')